In [1]:
pip install nibabel

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nibabel as nib
import random
import numpy as np
import os

img18_predict = nib.load('../18and20Output_predict4/sub018_seg_g0.01.nii.gz')
img20_predict = nib.load('../18and20Output_predict4/sub020_seg_g0.01.nii.gz')

img18_tta = nib.load('../18and20Output_tta4/sub018_seg_g0.01.nii.gz')
img20_tta = nib.load('../18and20Output_tta4/sub020_seg_g0.01.nii.gz')

img18_pre = nib.load('../18and20/sub018_seg_g0.01.nii.gz')
img20_pre = nib.load('../18and20/sub020_seg_g0.01.nii.gz')

data18_pre = img18_pre.get_fdata()
data20_pre = img20_pre.get_fdata()

data18_predict = img18_predict.get_fdata()
data20_predict = img20_predict.get_fdata()

data18_tta = img18_tta.get_fdata()
data20_tta = img20_tta.get_fdata()



In [6]:
#creates an image that highlights all voxels that are present in one input image, but not the other, and scales the image to standard intensities
#0: voxel is present in img2, but not img1
#255: voxel is present in img1, but not img2
#127.5: the same value in both images (either both 0 or both 1)
def create_diff_image(img1, img2, path):
    data1 = img1.get_fdata()
    data2 = img2.get_fdata()
    data_diff = data1 - data2
    affine = img1.affine
    img_min = np.min(data_diff)
    

    #scale image
    data_diff -= img_min
    print(np.min(data_diff))
    img_max = np.max(data_diff)
    data_diff *= (255/img_max)
    print(np.max(data_diff))

    diff_img = nib.Nifti1Image(data_diff, img1.affine, img1.header)
    nib.save(diff_img, path)


create_diff_image(img18_pre, img18_predict,'../18and20Output_predict4/18predict_diff.nii')
create_diff_image(img18_pre, img18_tta,'../18and20Output_tta4/18tta_diff.nii')

NameError: name 'np' is not defined

In [5]:
def compare_tta_pre_arrays(data1, data2, subject, comparison_type):
    mismatch_count = 0
    mismatches = []
    depth, height, width = data1.shape

    # Loop over each element in the 3D array
    for z in range(depth):
        for y in range(height):
            for x in range(width):
                # If the values are different, store the mismatch coordinates and values
                if data1[z, y, x] != data2[z, y, x]:
                    mismatches.append((z, y, x, data1[z, y, x], data2[z, y, x]))

    # Print the results
    for mismatch in mismatches:
        #print(f"Coordinates: {mismatch[:3]} - TTA: {mismatch[3]} - PRE: {mismatch[4]}")
        mismatch_count += 1
        
    return mismatch_count

def compare_predict_pre_arrays(data1, data2, subject, comparison_type):
    mismatch_count = 0
    mismatches = []
    depth, height, width = data1.shape

    # Loop over each element in the 3D array
    for z in range(depth):
        for y in range(height):
            for x in range(width):
                # If the values are different, store the mismatch coordinates and values
                if data1[z, y, x] != data2[z, y, x]:
                    mismatches.append((z, y, x, data1[z, y, x], data2[z, y, x]))

    # Print the results
    for mismatch in mismatches:
        #print(f"Coordinates: {mismatch[:3]} - Predict: {mismatch[3]} - PRE: {mismatch[4]}")
        mismatch_count += 1
        
    return mismatch_count

# List of subjects to loop through
subjects = ['018', '020']

# Loop over subjects to compare TTA and PREDICT with PRE
for subject in subjects:
    # Select the appropriate data arrays based on subject
    if subject == '018':
        data_pre = data18_pre
        data_predict = data18_predict
        data_tta = data18_tta
    elif subject == '020':
        data_pre = data20_pre
        data_predict = data20_predict
        data_tta = data20_tta

    # Compare TTA vs PRE
    countOfMismatchesTTA = compare_tta_pre_arrays(data_tta, data_pre, subject, "TTA vs PRE")
    print(f"Number of mismatches for TTA {subject} compared to the original {subject}: {countOfMismatchesTTA}")
    
    # Compare PREDICT vs PRE
    countOfMismatchesPredict = compare_predict_pre_arrays(data_predict, data_pre, subject, "PREDICT vs PRE")
    print(f"Number of mismatches for Predict {subject} compared to the original {subject}: {countOfMismatchesPredict}")
    

Number of mismatches for TTA 018 compared to the original 018: 108
Number of mismatches for Predict 018 compared to the original 018: 394630
Number of mismatches for TTA 020 compared to the original 020: 130298
Number of mismatches for Predict 020 compared to the original 020: 640677
